In [ ]:
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
import meshio
import torch
import torch.nn.functional as F
from torch.utils.data import random_split
from torch_geometric.loader import DataLoader
from nets import EncodeProcessDecode
from utils import info, msh_to_trimesh, visualize, normalize


# Train

In [ ]:
data = torch.load("data/thingi10k_1_1.pt", weights_only=False)
graphs = data["graphs"]

n = len(graphs)
n_train = int(1 * n)
n_val = n - n_train

print("Size of training set =", n_train)
print("Size of validation set =", n_val)

generator = torch.Generator().manual_seed(42)
train_graphs, val_graphs = map(
    list, random_split(graphs, [n_train, n_val], generator=generator)
)


In [ ]:
min(graphs[1].y)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

train_x = torch.cat([g.x for g in train_graphs], dim=0)
train_y = torch.cat([g.y for g in train_graphs], dim=0)
train_e = torch.cat([g.edge_attr for g in train_graphs], dim=0)
print("Training data shape:", train_x.shape, train_y.shape)

stats = {
    "x_mean": train_x.mean(dim=0),
    "x_std": train_x.std(dim=0),
    "y_mean": train_y.mean(dim=0),
    "y_std": train_y.std(dim=0),
    "e_mean": train_e.mean(dim=0),
    "e_std": train_e.std(dim=0),
}

print("x means:", stats["x_mean"])
print("x stds:", stats["x_std"])
print("y means:", stats["y_mean"])
print("y stds:", stats["y_std"])
print("Edge_attr means:", stats["e_mean"])
print("Edge_attr stds:", stats["e_std"])


train_loader = DataLoader(
    [normalize(g, stats) for g in train_graphs], batch_size=1, shuffle=False
)
val_loader = DataLoader(
    [normalize(g, stats) for g in val_graphs], batch_size=1, shuffle=False
)

In [ ]:
latent_dim = 128
node_dim, edge_dim, output_dim = info(graphs[0], debug=True)

In [ ]:
model = EncodeProcessDecode(
    node_dim=node_dim,
    edge_dim=edge_dim,
    output_dim=output_dim,
    latent_dim=latent_dim,
    message_passing_steps=15,
    use_layer_norm=True,
).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)

# for name, param in model.named_parameters():
#     print(f"{name}: {param.dtype}, shape: {param.shape}")

alpha = 0.1  # exponential scaling factor for mse loss
loss_history = []
num_epochs = 500
for epoch in range(num_epochs):
    model.train()
    total_loss = 0.0
    total_nodes = 0

    for batch in train_loader:
        batch = batch.to(device)
        optimizer.zero_grad()

        y_pred = model(batch)
        y_true = batch.y

        weight = torch.exp(-alpha * batch.x[:, 2].unsqueeze(1))
        weight = weight / weight.mean()

        loss = F.mse_loss(y_pred, y_true, weight=weight)
        loss.backward()
        optimizer.step()

        total_loss += loss.item() * batch.num_nodes
        total_nodes += batch.num_nodes
    avg_loss = total_loss / total_nodes
    loss_history.append(avg_loss)
    if (epoch + 1) % 50 == 0:
        print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {avg_loss:.6f}")

torch.save(
    {
        "model_state_dict": model.state_dict(),
        "params": {
            "node_dim": node_dim,
            "edge_dim": edge_dim,
            "output_dim": output_dim,
            "latent_dim": latent_dim,
            "message_passing_steps": 15,
            "use_layer_norm": True,
        },
        "stats": stats,
    },
    "models/model.pth",
)

In [ ]:
plt.plot(loss_history)
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Training Loss")
plt.show()

In [ ]:
model.eval()
total_loss = 0.0
total_nodes = 0

for batch in train_loader:
    batch = batch.to(device)
    optimizer.zero_grad()

    y_pred = model(batch)
    y_true = batch.y

    weight = (batch.x[:, 2].unsqueeze(1) < 1e-3).float()
    weight = weight / weight.mean()

    loss = F.mse_loss(y_pred, y_true, weight=weight)
    loss.backward()
    optimizer.step()

    total_loss += loss.item() * batch.num_nodes
    total_nodes += batch.num_nodes
avg_loss = total_loss / total_nodes
print("Validation loss =", avg_loss)

# Visualize Results

In [ ]:
import pyvista


pyvista.OFF_SCREEN = True

mesh = msh_to_trimesh(meshio.read("meshes/thingi10k/msh/42632.msh"))
g = train_graphs[2]
visualize(mesh, g, jupyter_backend="html", force_arrows=True)


In [ ]:
g_pred = g.clone()
g_pred.y = model(normalize(g, stats).to(device)).detach()
g_pred.y = g_pred.y * stats["y_std"].to(device) + stats["y_mean"].to(device)
visualize(mesh, g_pred, jupyter_backend="html", force_arrows=True)

## Example Graph

In [ ]:
num_nodes = 4
edge_index = torch.tensor(
    [[0, 1], [1, 0], [0, 2], [2, 0], [0, 3], [3, 0], [2, 3], [3, 2]]
).T
edge_attr = torch.tensor(
    [
        [3.0, 1.0],
        [-3.0, -1.0],
        [1.0, 2.0],
        [-1.0, -2.0],
        [2.0, 2.0],
        [-2.0, -2.0],
        [0.0, 3.0],
        [0.0, -3.0],
    ]
)

D = edge_attr.size(1)
idx = edge_index[1].unsqueeze(-1).expand(-1, D)
agg = torch.zeros((num_nodes, D))
deg = torch.zeros((num_nodes, 1))
deg.scatter_add_(
    0,
    edge_index[1].unsqueeze(-1),
    torch.ones((edge_index.size(1), 1)),
).clamp_(min=1.0)
agg.scatter_add_(0, idx, edge_attr)
print(agg)
print(deg)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)
graphs = torch.load("data/cantilever_1.pt", weights_only=False)
train_x = torch.cat([g["node"].x for g in graphs], dim=0)
train_y = torch.cat([g["node"].y for g in graphs], dim=0)
train_me = torch.cat([g["node", "mesh", "node"].edge_attr for g in graphs], dim=0)
train_ce = torch.cat([g["node", "contact", "node"].edge_attr for g in graphs], dim=0)
print("Training data shape:", train_x.shape, train_y.shape)

stats = {
    "x_mean": train_x.mean(dim=0),
    "x_std": train_x.std(dim=0),
    "y_mean": train_y.mean(dim=0),
    "y_std": train_y.std(dim=0),
    "me_mean": train_me.mean(dim=0),
    "me_std": train_me.std(dim=0),
    "ce_mean": train_ce.mean(dim=0),
    "ce_std": train_ce.std(dim=0),
}

print("x means:", stats["x_mean"])
print("x stds:", stats["x_std"])
print("y means:", stats["y_mean"])
print("y stds:", stats["y_std"])
print("Mesh edge_attr means:", stats["me_mean"])
print("Mesh edge_attr stds:", stats["me_std"])
print("Contact edge_attr means:", stats["ce_mean"])
print("Contact edge_attr stds:", stats["ce_std"])


def normalize_graph(graph, stats):
    # Clone to avoid overwriting original data in memory if needed
    g = graph.clone()

    # Handle std = 0 case by replacing with 1 (skips normalization for constant features)
    x_std_safe = torch.where(
        stats["x_std"][3:] > 0, stats["x_std"][3:], torch.ones_like(stats["x_std"][3:])
    )
    y_std_safe = torch.where(
        stats["y_std"] > 0, stats["y_std"], torch.ones_like(stats["y_std"])
    )
    me_std_safe = torch.where(
        stats["me_std"] > 0, stats["me_std"], torch.ones_like(stats["me_std"])
    )
    ce_std_safe = torch.where(
        stats["ce_std"] > 0, stats["ce_std"], torch.ones_like(stats["ce_std"])
    )

    g["node"].x[:, 3:] = (g["node"].x[:, 3:] - stats["x_mean"][3:]) / x_std_safe
    g["node"].y = (g["node"].y - stats["y_mean"]) / y_std_safe
    g["node", "mesh", "node"].edge_attr = (
        g["node", "mesh", "node"].edge_attr - stats["me_mean"]
    ) / me_std_safe
    g["node", "contact", "node"].edge_attr = (
        g["node", "contact", "node"].edge_attr - stats["ce_mean"]
    ) / ce_std_safe
    return g


loader = DataLoader(
    [normalize_graph(g, stats) for g in graphs], batch_size=1, shuffle=False
)

In [ ]:
import numpy as np

# Debugging: compare two graphs
g0 = graphs[0]
g1 = graphs[1]
x0 = g0.x
x1 = g1.x
dx = (x0 - x1).abs().max().item()
print("max |x0-x1| =", dx)

f0 = x0[:, -3:]
f1 = x1[:, -3:]
print("nonzero force nodes g0:", (f0.abs().sum(dim=1) > 0).sum().item())
print("nonzero force nodes g1:", (f1.abs().sum(dim=1) > 0).sum().item())

e0 = g0.edge_index
e1 = g1.edge_index
print("contact edges g0:", e0.size(1), "g1:", e1.size(1))
print("contact edge_index identical:", torch.equal(e0, e1))

idx0 = np.where(np.linalg.norm(x0[:, 3:6], axis=1) != 0)
idx1 = np.where(np.linalg.norm(x1[:, 3:6], axis=1) != 0)

print(idx0, idx1)
print(x0[idx0])
print(x1[idx1])